## Requirements


#### Directory adjustment


In [1]:
from pathlib import Path
import sys
import os

# Back to main folder
path = os.path.dirname(os.getcwd()) + "/"
os.chdir(path)
sys.path.append(path)


#### Charts


In [2]:
from IPython.display import SVG, display
import matplotlib.pyplot as plt
import seaborn as sns


/home/kunumi/miniconda3/envs/hate-seepch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Data Processing


In [3]:
# ETL
import numpy as np
import pandas as pd

# ML preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.feature_selection import chi2


In [4]:
from gensim.models import Doc2Vec


#### Natural language processing


In [5]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import gensim


In [6]:
from src.TextVectorization import MeanEmbeddingVectorizer

# import spacy
import re

# Vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


#### Models


In [7]:
# Pipe
from sklearn.pipeline import Pipeline

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn import svm


# Metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


In [8]:
from gensim.models import KeyedVectors
from gensim import models
import gensim


In [9]:
# class MeanEmbeddingVectorizer(object):
#     def __init__(self, word2vec):
#         self.word2vec = word2vec
#         # if a text is empty we should return a vector of zeros
#         # with the same dimensionality as all the other vectors
#         self.dim = int(np.mean([len(v) for v in word2vec.values()]))

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         return np.array(
#             [
#                 np.mean(
#                     [self.word2vec[w] for w in words if w in self.word2vec]
#                     or [np.zeros(self.dim)],
#                     axis=0,
#                 )
#                 for words in X
#             ]
#         )


## Set and split train and test data


In [10]:
# Get data
df = pd.read_csv("data/augmented_corpus_fortuna.csv")
df.head(2)


,text_nonstop,text_lemma,text,length_text_nonstop,length_text_lemma,length_text,label,count_word_text_nonstop,count_word_text_lemma,count_word_text,...,pron,adp,aux,cconj,num,space,intj,sym,punct,part
0,cara vive outro mundo nao mundo real refugiado...,caro viver outro mundo nao mundo real refugiad...,nomeusuario o cara vive em outro mundo nao no ...,85,82,124,1,19,19,20,...,0,0,0,0,0,0,0,0,0,0
1,incompetentes nao cuidam povo brasileiro pouco...,incompetente nao cuidar povo brasileiro pouco ...,nomeusuario estes incompetentes nao cuidam nem...,69,66,108,0,20,20,20,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Set target and features
target = "label"
features = "text_nonstop"

# Break apart dataset
X = df[features].values.astype("U")
y = df[target]

# Split train abd test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Set k-fold criteria
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)


## Word2Vec 


In [12]:
# Define corpus
corpus = X


In [13]:
# Train a own word2vec model
own_model = gensim.models.Word2Vec(
    corpus, vector_size=50, window=8, min_count=5, sg=1, workers=4, batch_words=10000
)

own_w2v = dict(zip(own_model.wv.index_to_key, own_model.wv.vectors))


In [14]:
# Load a pre-trained model
pretrained_model = models.KeyedVectors.load_word2vec_format(
    "data/skip_s50.txt", binary=False
)
pretrained_w2v = dict(zip(pretrained_model.index_to_key, pretrained_model.vectors))


In [15]:
# Basic pipeline
clf_own = Pipeline(
    [
        ("word2vec vectorizer", MeanEmbeddingVectorizer(own_w2v)),
        (
            "extra trees",
            DecisionTreeClassifier(random_state=42, class_weight={0: 1, 1: 1.5}),
        ),
    ]
)

# Train
clf_own.fit(X_train, y_train)

pd.DataFrame(classification_report(y_test, clf_own.predict(X_test), output_dict=True)).T


,precision,recall,f1-score,support
0,0.719737,0.703990,0.711776,777.000000
1,0.385027,0.403361,0.393981,357.000000
accuracy,0.609347,0.609347,0.609347,0.609347
macro avg,0.552382,0.553676,0.552879,1134.000000
weighted avg,0.614365,0.609347,0.611730,1134.000000


In [16]:
# Basic pipeline
clf_pretrained = Pipeline(
    [
        ("word2vec vectorizer", MeanEmbeddingVectorizer(pretrained_w2v)),
        (
            "extra trees",
            DecisionTreeClassifier(random_state=42, class_weight={0: 1, 1: 1.5}),
        ),
    ]
)

# Train
clf_pretrained.fit(X_train, y_train)

pd.DataFrame(
    classification_report(y_test, clf_pretrained.predict(X_test), output_dict=True)
).T



,precision,recall,f1-score,support
0,0.725032,0.719434,0.722222,777.000000
1,0.399449,0.406162,0.402778,357.000000
accuracy,0.620811,0.620811,0.620811,0.620811
macro avg,0.562241,0.562798,0.562500,1134.000000
weighted avg,0.622534,0.620811,0.621656,1134.000000
